# Crop simulation

Doing imports 

In [22]:
%matplotlib inline
#%matplotlib notebook
import sys
import pcse
import pandas as pd 
import numpy as np
import matplotlib
matplotlib.style.use("ggplot")
import matplotlib.pyplot as plt
print("This notebook was built with:")
print(f"python version: {sys.version}")
print(f"PCSE version: {pcse.__version__}")

This notebook was built with:
python version: 3.10.16 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:19:12) [MSC v.1929 64 bit (AMD64)]
PCSE version: 6.0.9


## Transform the data into the right units


| Identifier | Parameter   | Description                              | From Unit  | To Unit           | Dataset |
|------------|-------------|------------------------------------------|------------|-------------------|---------|
| TXK        | TMAX        | Daily maximum temperature                | °C         | °C                | KL      |
| TNK        | TMIN        | Daily minimum temperature                | °C         | °C                | KL      |
| VPM        | VAP         | Mean daily vapour pressure               | hPa        | hPa               | KL      |
| FM         | WIND        | Mean daily wind speed at 2 m above ground level | m/s        | m/s       | KL      |
| RSK        | RAIN        | Precipitation                            | mm         | cm   | KL      |
| FG_STRAHL  | IRRAD       | Daily global radiation                   | J/cm^2     | J/m^{-2} | Solar   |
| SHK_TAG    | SNOWDEPTH   | Depth of snow cover (optional)           | cm         | cm                | KL      |

#### Climate data

In [ ]:
# Read the text file into a DataFrame
data_bremen_kl = pd.read_csv('data_kl/bremen_daily_kl/tageswerte_KL_00691_18900101_20231231_hist/produkt_klima_tag_18900101_20231231_00691.txt', delimiter=';', skipfooter=1, engine='python')

data_chemnitz_kl = pd.read_csv('data_kl/chemnitz_daily_kl/tageswerte_KL_00853_18820101_20231231_hist/produkt_klima_tag_18820101_20231231_00853.txt', delimiter=';', skipfooter=1, engine='python')

# Convert units -> mm to cm, ensuring column names are stripped of extra spaces
data_bremen_kl.columns = data_bremen_kl.columns.str.strip()
data_chemnitz_kl.columns = data_chemnitz_kl.columns.str.strip()

# Replace -999 with NaN
data_bremen_kl.replace(-999, np.nan, inplace=True)
data_chemnitz_kl.replace(-999, np.nan, inplace=True)

# Remove rows with any NaN values
# data_bremen_kl.dropna(inplace=True)
# data_chemnitz_kl.dropna(inplace=True)

#### Temperature data

In [ ]:
# Read the text file into a DataFrame
data_bremen_solar = pd.read_csv('data_solar/bremen/tageswerte_ST_00691_row/produkt_st_tag_20090101_20250331_00691.txt', delimiter=';', skipfooter=1, engine='python')

data_chemnitz_solar = pd.read_csv('data_solar/chemnitz/tageswerte_ST_00853_row/produkt_st_tag_19810101_20250331_00853.txt', delimiter=';', skipfooter=1, engine='python')

# Replace -999 with NaN
data_bremen_solar.replace(-999, np.nan, inplace=True)
data_chemnitz_solar.replace(-999, np.nan, inplace=True)

# Remove rows with any NaN values
# data_bremen_solar.dropna(inplace=True)
# data_chemnitz_solar.dropna(inplace=True)

### Transform to PCSE data format

In [47]:
# Map and transform the data to PCSE format
# Example mapping (adjust according to your actual data):
data_bremen_pcse = pd.DataFrame({
    #'DAY': pd.to_datetime(data_bremen_kl['MESS_DATUM'], format='%Y%m%d'),
    'DAY': data_bremen_kl['MESS_DATUM'],
    'TMIN': data_bremen_kl['TNK'],  # Example: map to minimum temperature
    'TMAX': data_bremen_kl['TXK'],  # Example: map to maximum temperature
    'VAP': data_bremen_kl['VPM'],  # Example: map to vapor pressure
    'RAIN': data_bremen_kl['RSK'] / 10,  # Convert mm to cm if needed
    'IRRAD': data_bremen_solar['FG_STRAHL'] * 10000,  # Convert J/cm² to J/m²
    'WIND': data_bremen_kl['FM'],  # Assuming this is the wind speed
    'Angstrom_A': 0.25,  # Default Angstrom A value
    'Angstrom_B': 0.50   # Default Angstrom B value
    # Add other required fields as necessary
})

print(data_bremen_pcse)

            DAY  TMIN  TMAX  VAP  RAIN      IRRAD  WIND  Angstrom_A  \
0      18900101  -5.5  -1.4  4.4  0.02  2660000.0   NaN        0.25   
1      18900102  -7.4  -3.3  4.1  0.00  2610000.0   NaN        0.25   
2      18900103  -3.5  -1.3  5.2  0.00  1370000.0   NaN        0.25   
3      18900104  -1.9   5.7  6.5  0.00   720000.0   NaN        0.25   
4      18900105   0.2   7.7  7.2  0.11  1870000.0   NaN        0.25   
...         ...   ...   ...  ...   ...        ...   ...         ...   
48630  20231226   2.3   9.7  8.8  0.04        NaN   8.0        0.25   
48631  20231227   0.3   8.7  7.4  0.19        NaN   3.8        0.25   
48632  20231228   7.9  11.1  9.2  0.09        NaN   9.4        0.25   
48633  20231229   5.4  10.8  9.1  1.13        NaN   8.4        0.25   
48634  20231230   5.4   7.8  8.2  0.00        NaN   6.0        0.25   

       Angstrom_B  
0             0.5  
1             0.5  
2             0.5  
3             0.5  
4             0.5  
...           ...  
48630  

In [48]:
# Map and transform the data to PCSE format
# Example mapping (adjust according to your actual data):
data_chemnitz_pcse = pd.DataFrame({
    #'DAY': pd.to_datetime(data_chemnitz_kl['MESS_DATUM'], format='%Y%m%d'),
    'DAY': data_chemnitz_kl['MESS_DATUM'],
    'TMIN': data_chemnitz_kl['TNK'],  # Example: map to minimum temperature
    'TMAX': data_chemnitz_kl['TXK'],  # Example: map to maximum temperature
    'VAP': data_chemnitz_kl['VPM'],  # Example: map to vapor pressure
    'RAIN': data_chemnitz_kl['RSK'] / 10,  # Convert mm to cm if needed
    'IRRAD': data_chemnitz_solar['FG_STRAHL'] * 10000,  # Convert J/cm² to J/m²
    'WIND': data_chemnitz_kl['FM'],  # Assuming this is the wind speed
    # Add other required fields as necessary
})

print(data_chemnitz_pcse[14000:15000])

            DAY  TMIN  TMAX   VAP  RAIN       IRRAD  WIND
14000  19390602   8.4  20.1  12.0   0.0  23920000.0   NaN
14001  19390603   6.0  20.2   9.1   0.0   8760000.0   NaN
14002  19390604   6.5  21.4  11.5   0.0   7010000.0   NaN
14003  19390605  11.6  21.9  12.5   0.0  14700000.0   NaN
14004  19390606  12.0  25.6  12.0   0.0  14310000.0   NaN
...         ...   ...   ...   ...   ...         ...   ...
14995  19420221 -16.2  -3.7   2.8   0.0   4190000.0   NaN
14996  19420222  -7.4   0.3   3.5   0.0   2760000.0   NaN
14997  19420223  -4.5   6.1   4.3   0.0   1310000.0   NaN
14998  19420224  -4.4  -1.8   4.8   0.0   1920000.0   NaN
14999  19420225  -4.0  -1.5   4.9   0.0    670000.0   NaN

[1000 rows x 7 columns]


In [27]:
from pcse.fileinput import ExcelWeatherDataProvider
# Ensure the file is properly created as an Excel file
data_bremen_pcse.to_excel("bremen_daily.xlsx", index=False)

# Load the weather data provider
weatherfile = "bremen_daily.xlsx"
weatherdataprovider = ExcelWeatherDataProvider(weatherfile)
print(weatherdataprovider)

PermissionError: [Errno 13] Permission denied: 'bremen_daily.xlsx'

In [ ]:
#crop, soil, site and agro
from pcse.util import DummySoilDataProvider
from pcse.input import WOFOST72SiteDataProvider 
import yaml
from pcse.base import ParameterProvider

#load crop file
with open('data/crop/potato.yaml', 'r') as file: #alternative: from pcse.input import YAMLCropDataProvider
    cropd = yaml.safe_load(file)

#load dummy soil file
soild = DummySoilDataProvider() #use dummy soil: https://pcse.readthedocs.io/en/stable/code.html#pcse.util.DummySoilDataProvider

#load some standard site
sited = WOFOST72SiteDataProvider(WAV=10) #https://pcse.readthedocs.io/en/stable/code.html#wofost72sitedataprovider

#put everything together with ParameterProvider
parameters = ParameterProvider(cropdata=cropd, soildata=soild, sitedata=sited)

#agro file (is not completed yet)
with open('data/agro/potato_agro_test.yaml', 'r') as file:
    agromanagement = yaml.safe_load(file)